How many alignments does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [ ]:
def naive_exact_match(pattern, text):
    alignments = 0
    pattern_length = len(pattern)
    text_length = len(text)

    for i in range(text_length - pattern_length + 1):
        alignments += 1
        if text[i:i+pattern_length] == pattern:
            print("Alignment found at index:", i)

    return alignments

# Define the pattern and text
pattern = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    text = ''.join(line.strip() for line in lines)

# Count alignments
alignments = naive_exact_match(pattern, text)
print("Total alignments tried:", alignments)


Alignment found at index: 56922
Total alignments tried: 799954


2. How many character comparisons does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.

In [ ]:
def naive_exact_match(pattern, text):
    comparisons = 0
    alignments = 0
    pattern_length = len(pattern)
    text_length = len(text)

    for i in range(text_length - pattern_length + 1):
        alignments += 1
        for j in range(pattern_length):
            comparisons += 1
            if text[i+j] != pattern[j]:
                break

    return comparisons

# Define the pattern and text
pattern = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    text = ''.join(line.strip() for line in lines)

# Count character comparisons
comparisons = naive_exact_match(pattern, text)
print("Total character comparisons tried:", comparisons)



Total character comparisons tried: 984143


3.How many alignments does Boyer-Moore try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [ ]:
def boyer_moore_with_count(p, p_bm, t):
    """ Do Boyer-Moore matching """
    occurrences = []
    alignments_tried = 0
    i = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        alignments_tried += 1
        for j in range(len(p) - 1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, alignments_tried

from bm_preproc import BoyerMoore

# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    t = ''.join(line.strip() for line in lines)

# Define the pattern
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"

# Preprocess the pattern for Boyer-Moore
p_bm = BoyerMoore(p)

# Perform Boyer-Moore matching with count
occurrences, alignments_tried = boyer_moore_with_count(p, p_bm, t)

print("Total alignments tried by Boyer-Moore:", alignments_tried)


Total alignments tried by Boyer-Moore: 127974


In [ ]:
import bisect

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def approximate_match_index(p, t, index, k):
    segment_length = len(p) // 3
    all_matches = set()
    hits = 0
    for i in range(3):
        start = i * segment_length
        end = (i + 1) * segment_length
        matches = index.query(p[start:end])
        hits += len(matches)
        for match in matches:
            if match < start or match - start + len(p) > len(t):
                continue
            mismatches = 0
            for j in range(start):
                if p[j] != t[match - start + j]:
                    mismatches += 1
                    if mismatches > 2:
                        break
            for j in range(end, len(p)):
                if p[j] != t[match - start + j]:
                    mismatches += 1
                    if mismatches > 2:
                        break
            if mismatches <= 2:
                all_matches.add(match - start)
    return list(all_matches), hits

# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    text = ''.join(line.strip() for line in lines)

# Create the index
from kmer_index import Index
index = Index(text, 8)

# Define the pattern
pattern = "GGCGCGGTGGCTCACGCCTGTAAT"

# Find approximate matches using the index
matches, hits = approximate_match_index(pattern, text, index, 8)
print("Total approximate matches found:", len(matches))


Total approximate matches found: 19


In [ ]:
# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    text = ''.join(line.strip() for line in lines)

# Create the index
from kmer_index import Index
index = Index(text, 8)

# Define the pattern
pattern = "GGCGCGGTGGCTCACGCCTGTAAT"

# Find approximate matches using the index
matches, hits = approximate_match_index(pattern, text, index, 8)
print("Total index hits:", hits)


Total index hits: 90


In [ ]:
import bisect

class SubseqIndex(object):
    """ Holds a subsequence index for a text T """

    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq

    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def count_mismatches(seq1, seq2):
    """ Count the number of mismatches between two sequences """
    return sum(1 for base1, base2 in zip(seq1, seq2) if base1 != base2)

def approximate_match_subseq_index(pattern, text, index):
    """ Find all approximate occurrences of pattern in text using SubseqIndex """
    k = index.k
    ival = index.ival
    hits = set()

    # Find hits for first subsequence of pattern
    subseq_hits = index.query(pattern)
    for hit in subseq_hits:
        if hit not in hits:
            hits.add(hit)

    # Check remaining subsequences of pattern
    for i in range(1, (len(pattern) - k) // ival + 1):
        subseq = pattern[i * ival:i * ival + k]
        subseq_hits = index.query(subseq)
        for hit in subseq_hits:
            if hit - i * ival not in hits:
                hits.add(hit - i * ival)

    # Check for mismatches
    matches = []
    for hit in hits:
        if count_mismatches(text[hit:hit+len(pattern)], pattern) <= 2:
            matches.append(hit)

    return matches

# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    text = ''.join(line.strip() for line in lines)

# Define the pattern
pattern = "GGCGCGGTGGCTCACGCCTGTAAT"

# Create SubseqIndex
subseq_index = SubseqIndex(text, 8, 3)

# Find approximate matches using SubseqIndex
matches = approximate_match_subseq_index(pattern, text, subseq_index)
print("Total index hits for approximate matches:", len(matches))


Total index hits for approximate matches: 13


What is the edit distance of the best match between pattern
GCTGATCGATCGTACG
GCTGATCGATCGTACG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [ ]:
def edit_distance(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    t = ''.join(line.strip() for line in lines)

# Define the pattern
p = "GCTGATCGATCGTACG"

# Find the best match and calculate its edit distance
best_match = min([edit_distance(p, t[i:i+len(p)]) for i in range(len(t) - len(p) + 1)])
print("Edit distance of the best match:", best_match)


Edit distance of the best match: 3


What is the edit distance of the best match between pattern
GATTTACCAGATTGAG
GATTTACCAGATTGAG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [ ]:
def edit_distance(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

# Load the excerpt of human chromosome 1
with open("chr1.GRCh38.excerpt.fasta") as f:
    lines = f.readlines()[1:]
    t = ''.join(line.strip() for line in lines)

# Define the pattern
p = "GATTTACCAGATTGAG"

# Find the best match and calculate its edit distance
best_match = min([edit_distance(p, t[i:i+len(p)]) for i in range(len(t) - len(p) + 1)])
print("Edit distance of the best match:", best_match)


Edit distance of the best match: 2


In [ ]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def find_suffix_prefix_matches(reads, min_length=30):
    suffixes = {}
    overlaps = set()

    # Step 1: Create a dictionary where each key is a length-30 suffix of a read
    for read in reads:
        suffix = read[-min_length:]
        if suffix not in suffixes:
            suffixes[suffix] = set()
        suffixes[suffix].add(read)

    # Step 2: Find overlaps involving each read's suffix
    for read in reads:
        suffix = read[-min_length:]
        if suffix in suffixes:
            for other_read in suffixes[suffix]:
                if other_read != read:
                    olen = overlap(read, other_read, min_length)
                    if olen >= min_length:
                        overlaps.add((read, other_read))

    return overlaps

# Step 1: Download and parse the reads from the provided FASTQ file
reads = []
with open("ERR266411_1.for_asm.fastq") as f:
    for line in f:
        if not line.startswith("@"):  # Ignore read names
            reads.append(line.strip())

# Step 2: Find all pairs of reads with an exact suffix/prefix match of length at least 30
matches = find_suffix_prefix_matches(reads)

# Step 3: Count the number of distinct pairs of reads that overlap
num_edges = len(matches)
print("Number of edges in the overlap graph:", num_edges)


Number of edges in the overlap graph: 0


In [ ]:
# Step 1: Download and parse the reads from the provided FASTQ file
reads = []
with open("ERR266411_1.for_asm.fastq") as f:
    for line in f:
        if not line.startswith("@"):  # Ignore read names
            reads.append(line.strip())

# Step 2: Find all pairs of reads with an exact suffix/prefix match of length at least 30
matches = find_suffix_prefix_matches(reads)

# Step 3: Extract all unique reads involved in overlaps
nodes_with_edges = set()
for read_pair in matches:
    nodes_with_edges.add(read_pair[0])
    nodes_with_edges.add(read_pair[1])

# Step 4: Count the number of nodes with at least one outgoing edge
num_nodes_with_edges = len(nodes_with_edges)
print("Number of nodes with at least one outgoing edge:", num_nodes_with_edges)


Number of nodes with at least one outgoing edge: 0


In [ ]:
# Step 1: Download and parse the reads from the provided FASTQ file
reads = []
with open("ERR266411_1.for_asm.fastq") as f:
    for line in f:
        if not line.startswith("@"):  # Ignore read names
            reads.append(line.strip())

# Step 2: Find all pairs of reads with an exact suffix/prefix match of length at least 30
matches = find_suffix_prefix_matches(reads)

# Step 3: Extract all unique reads involved in overlaps
nodes_with_edges = set()
for read_pair in matches:
    nodes_with_edges.add(read_pair[0])
    nodes_with_edges.add(read_pair[1])

# Step 4: Count the number of nodes with at least one outgoing edge
num_nodes_with_edges = len(nodes_with_edges)
print("Number of nodes with at least one outgoing edge:", num_nodes_with_edges)


Number of nodes with at least one outgoing edge: 0


In [ ]:
# Check if reads are correctly parsed
print("Number of reads:", len(reads))
print("Example reads:")
for read in reads[:5]:
    print(read)


Number of reads: 28865
Example reads:
TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC
+
B@DFEFFFGEGGGHEHGHGHGGGGHIFGFIFHICFGHGHGJGHFGHGIHEHGGHJGFEFHGHEGGHHGHIFGFGDIFGGFGGGFHGGGHGGGAGIFGGCG
AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC
+


In [ ]:
# Test the overlap function
overlap_length1 = overlap("TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC", "AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC")
print("Overlap length:", overlap_length1)

overlap_length2 = overlap("TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC", "CGATAAAACTCTCAAGTTGCTTTTCTTCAGCTTGGCGGAGAAGTCAAGTAACTTGGCCGGGGTCGTTTGCTGGCACATACCCATGCAAGCGTGAAACTT")
print("Overlap length:", overlap_length2)


Overlap length: 0
Overlap length: 11


In [ ]:
from collections import defaultdict

def parse_fastq(filename):
    """Parse a FASTQ file and return a list of reads."""
    reads = []
    with open(filename, 'r') as file:
        for line in file:
            # Skip description lines
            if line[0] == '@':
                next(file)
                reads.append(next(file).strip())
    return reads

def get_kmers(read, k):
    """Generate all k-mers of length k from a read."""
    return [read[i:i+k] for i in range(len(read) - k + 1)]

def build_kmer_index(reads, k):
    """Build a dictionary where each k-mer is associated with a set of reads containing that k-mer."""
    kmer_index = defaultdict(set)
    for read_id, read in enumerate(reads):
        for kmer in get_kmers(read, k):
            kmer_index[kmer].add(read_id)
    return kmer_index

def find_overlaps(reads, k, min_length):
    """Find all pairs of reads with an exact suffix/prefix match of length at least min_length."""
    kmer_index = build_kmer_index(reads, k)
    overlap_pairs = set()
    for read_id, read in enumerate(reads):
        suffix = read[-k:]
        for other_read_id in kmer_index[suffix]:
            if read_id != other_read_id:
                other_read = reads[other_read_id]
                overlap_length = overlap(read, other_read, min_length)
                if overlap_length >= min_length:
                    overlap_pairs.add((read_id, other_read_id))
    return overlap_pairs

# Parse the FASTQ file
filename = "ERR266411_1.for_asm.fastq"
reads = parse_fastq(filename)

# Find overlaps
k = 6
min_length = 30
overlap_pairs = find_overlaps(reads, k, min_length)

# Count the number of distinct pairs of reads that overlap
num_edges = len(overlap_pairs)
print("Number of edges in the overlap graph:", num_edges)


Number of edges in the overlap graph: 12114


In [ ]:
from collections import defaultdict

def parse_fastq(filename):
    """Parse a FASTQ file and return a list of reads."""
    reads = []
    with open(filename, 'r') as file:
        for line in file:
            # Skip description lines
            if line[0] == '@':
                next(file)
                reads.append(next(file).strip())
    return reads

def overlap(a, b, min_length=3):
    """Return length of longest suffix of 'a' matching a prefix of 'b' that is at least 'min_length' characters long."""
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def get_kmers(read, k):
    """Generate all k-mers of length k from a read."""
    return [read[i:i+k] for i in range(len(read) - k + 1)]

def build_kmer_index(reads, k):
    """Build a dictionary where each k-mer is associated with a set of reads containing that k-mer."""
    kmer_index = defaultdict(set)
    for read_id, read in enumerate(reads):
        for kmer in get_kmers(read, k):
            kmer_index[kmer].add(read_id)
    return kmer_index

def find_overlaps(reads, k, min_length):
    """Find all pairs of reads with an exact suffix/prefix match of length at least min_length."""
    kmer_index = build_kmer_index(reads, k)
    overlap_pairs = set()
    for read_id, read in enumerate(reads):
        suffix = read[-k:]
        for other_read_id in kmer_index[suffix]:
            if read_id != other_read_id:
                other_read = reads[other_read_id]
                overlap_length = overlap(read, other_read, min_length)
                if overlap_length >= min_length:
                    overlap_pairs.add((read_id, other_read_id))
    return overlap_pairs

def build_overlap_graph(reads, k, min_length):
    """Build an overlap graph where each read is a node and edges represent overlaps."""
    overlap_graph = defaultdict(set)
    kmer_index = build_kmer_index(reads, k)
    for read_id, read in enumerate(reads):
        suffix = read[-k:]
        for other_read_id in kmer_index[suffix]:
            if read_id != other_read_id:
                other_read = reads[other_read_id]
                overlap_length = overlap(read, other_read, min_length)
                if overlap_length >= min_length:
                    overlap_graph[read_id].add(other_read_id)
    return overlap_graph

# Parse the FASTQ file
filename = "ERR266411_1.for_asm.fastq"
reads = parse_fastq(filename)

# Build the overlap graph
k = 6
min_length = 30
overlap_graph = build_overlap_graph(reads, k, min_length)

# Count the number of nodes with at least one outgoing edge
num_nodes_with_outgoing_edge = sum(1 for node_edges in overlap_graph.values() if len(node_edges) > 0)
print("Number of nodes with at least one outgoing edge:", num_nodes_with_outgoing_edge)


Number of nodes with at least one outgoing edge: 623
